In [2]:
import pandas as pd
import numpy as np
import requests
import time
import datetime

In [5]:
token = open('C:/Users/Danila/Desktop/jupyter/my/Тинькофф инвестиции/Проект 1.txt', 'r').read()

In [3]:
def money(v):
    return int(v['units']) + int(v['nano']) / 1e9

In [4]:
def get_bonds():
    headers = {
    'accept': 'application/json',
    'Authorization': 'Bearer '+token,
    'Content-Type': 'application/json',
}

    json_data = {
    'instrumentStatus': 'INSTRUMENT_STATUS_ALL',
}

    response = requests.post(
    'https://invest-public-api.tinkoff.ru/rest/tinkoff.public.invest.api.contract.v1.InstrumentsService/Bonds',
    headers=headers,
    json=json_data,
)
    figi_b = []
    ticker_b = []
    nominal_b = []
    risk_level_b = []
    currency_b = []

    for i in response.json()['instruments']:
        if i['otcFlag'] == False:
            figi_b.append(i['figi'])
            ticker_b.append(i['ticker'])
            nominal_b.append(money(i['nominal']))
            risk_level_b.append(i['riskLevel'])
            currency_b.append(i['currency'])
    
    all_bonds = pd.DataFrame({'figi':figi_b,
                              'ticker':ticker_b,
                              'nominal':nominal_b,
                              'risk_level':risk_level_b,
                              'currency':currency_b})
    return all_bonds

In [5]:
def get_coupons(figi, date_start, date_end):
    headers = {
    'accept': 'application/json',
    'Authorization': 'Bearer '+token,    
    'Content-Type': 'application/json',
}

    json_data = {
    'figi': figi,
    'from': date_start+'T00:00:00.000Z',
    'to': date_end+'T00:00:00.000Z',
}

    response = requests.post(
    'https://invest-public-api.tinkoff.ru/rest/tinkoff.public.invest.api.contract.v1.InstrumentsService/GetBondCoupons',
    headers=headers,
    json=json_data,
)
    figi_b = []
    coup_data = []
    coup_fix = []
    coup_pay = []
    
    if response.status_code == 429:
        time.sleep(60)
    else:
        for i in response.json()['events']:
            figi_b.append(i['figi'])
            coup_data.append(i['couponDate'][:10])
            try:
                coup_fix.append(i['fixDate'][:10])
            except LookupError:
                coup_fix.append(i['couponDate'][:10])
            coup_pay.append(money(i['payOneBond']))
            
    return figi_b, coup_data, coup_fix, coup_pay

In [6]:
def info_coupons(list_figi_bonds_for_info, date_start, date_end):
    figi_all = []
    data_all = []
    fix_all = []
    pay_all = []
    for figi in list_figi_bonds_for_info:
        gc1,gc2,gc3,gc4 = get_coupons(figi, date_start, date_end)
        figi_all += gc1
        data_all += gc2
        fix_all += gc3
        pay_all += gc4
    return pd.DataFrame({'figi':figi_all,
                         'data': data_all,
                         'fix': fix_all,
                         'pay': pay_all})

In [7]:
def get_candles_day(figi, date_start, date_end_v=None):
    if date_end_v == None or date_start == date_end_v:
        date_end = str((datetime.datetime.strptime(date_start, '%Y-%m-%d')+datetime.timedelta(days=1)).date())
    else:
        date_end = date_end_v

    headers = {
    'accept': 'application/json',
    'Authorization': 'Bearer '+token,
    'Content-Type': 'application/json',
}

    json_data = {
    'from': date_start+'T00:00:00.000Z',
    'to': date_end+'T00:00:00.000Z',
    'interval': 'CANDLE_INTERVAL_DAY',
    'instrumentId': figi,
}

    response = requests.post(
    'https://invest-public-api.tinkoff.ru/rest/tinkoff.public.invest.api.contract.v1.MarketDataService/GetCandles',
    headers=headers,
    json=json_data,
)
    
    open_c = []
    high_c = []
    low_c = []
    close_c = []
    time_c = []
    
    if response.status_code == 429:
        time.sleep(60)
    else:
        for candles in response.json()['candles']:
            open_c.append(money(candles['open']))
            high_c.append(money(candles['high']))
            low_c.append(money(candles['low']))
            close_c.append(money(candles['close']))
            time_c.append(candles['time'][:10])
        
    if date_end_v == None or date_start == date_end_v:
        return [figi] * len(open_c[:1]), time_c[:1], open_c[:1], high_c[:1], low_c[:1], close_c[:1] 
    else:
        return [figi] * len(open_c), time_c, open_c, high_c, low_c, close_c

In [8]:
def info_bonds_hist_price(list_figi_bonds_for_info, date_start, date_end=None):
    figi_all = []
    open_all = []
    high_all = []
    low_all = []
    close_all = []
    data_all = []
    
    for figi in list_figi_bonds_for_info:
        gca1,gca2,gca3,gca4,gca5,gca6  = get_candles_day(figi, date_start, date_end)
        if len(gca1) > 0:
            figi_all += gca1
            data_all += gca2
            open_all += gca3
            high_all += gca4
            low_all += gca5
            close_all += gca6
            
    return pd.DataFrame({'figi':figi_all,
                         'data_candles': data_all,
                         'open': open_all,
                         'high': high_all,
                         'low':low_all,
                         'close': close_all})

In [9]:
bonds = get_bonds()
bonds_rub_less_1000 = bonds[(bonds['nominal']<=1000) & (bonds['currency']=='rub') & (bonds['nominal']>0)]
bonds_rub_less_1000

,figi,ticker,nominal,risk_level,currency
0,BBG00XH4W3N3,RU000A101RZ3,1000.00,RISK_LEVEL_LOW,rub
2,TCS00A102ZS9,RU000A102ZS9,1000.00,RISK_LEVEL_HIGH,rub
3,BBG00Z8KGGC3,RU000A102R73,1000.00,RISK_LEVEL_LOW,rub
4,TCS00A102D46,RU000A102D46,518.11,RISK_LEVEL_LOW,rub
5,BBG0116Q1YF0,RU000A103455,1000.00,RISK_LEVEL_UNSPECIFIED,rub
...,...,...,...,...,...
1792,TCS00A105HJ9,RU000A105HJ9,1000.00,RISK_LEVEL_UNSPECIFIED,rub
1793,BBG00T7HZJG4,RU000A101L96,1000.00,RISK_LEVEL_MODERATE,rub
1794,TCS00A105DS9,RU000A105DS9,1000.00,RISK_LEVEL_MODERATE,rub
1795,TCS00A104735,RU000A104735,1000.00,RISK_LEVEL_MODERATE,rub


In [10]:
df_coupons = info_coupons(list(bonds_rub_less_1000['figi']), '2023-04-01', '2024-04-01')
df_coupons

,figi,data,fix,pay
0,BBG00XH4W3N3,2024-03-20,2024-03-19,34.90
1,BBG00XH4W3N3,2023-09-20,2023-09-19,34.90
2,TCS00A102ZS9,2024-01-09,2023-12-29,29.92
3,TCS00A102ZS9,2023-10-10,2023-10-09,29.92
4,TCS00A102ZS9,2023-07-11,2023-07-10,29.92
...,...,...,...,...
3865,TCS00A104735,2023-12-07,2023-12-06,29.29
3866,TCS00A104735,2023-09-07,2023-09-06,29.29
3867,TCS00A104735,2023-06-08,2023-06-07,29.29
3868,BBG00J7HHGH1,2023-11-14,2023-11-13,45.87


In [11]:
df_coupons_p = df_coupons[df_coupons['pay']>0]
df_coupons_p

,figi,data,fix,pay
0,BBG00XH4W3N3,2024-03-20,2024-03-19,34.90
1,BBG00XH4W3N3,2023-09-20,2023-09-19,34.90
2,TCS00A102ZS9,2024-01-09,2023-12-29,29.92
3,TCS00A102ZS9,2023-10-10,2023-10-09,29.92
4,TCS00A102ZS9,2023-07-11,2023-07-10,29.92
...,...,...,...,...
3865,TCS00A104735,2023-12-07,2023-12-06,29.29
3866,TCS00A104735,2023-09-07,2023-09-06,29.29
3867,TCS00A104735,2023-06-08,2023-06-07,29.29
3868,BBG00J7HHGH1,2023-11-14,2023-11-13,45.87


In [12]:
bonds_pay_info = bonds_rub_less_1000.merge(df_coupons_p,
                             how='inner',
                             on='figi')
bonds_pay_info

,figi,ticker,nominal,risk_level,currency,data,fix,pay
0,BBG00XH4W3N3,RU000A101RZ3,1000.0,RISK_LEVEL_LOW,rub,2024-03-20,2024-03-19,34.90
1,BBG00XH4W3N3,RU000A101RZ3,1000.0,RISK_LEVEL_LOW,rub,2023-09-20,2023-09-19,34.90
2,TCS00A102ZS9,RU000A102ZS9,1000.0,RISK_LEVEL_HIGH,rub,2024-01-09,2023-12-29,29.92
3,TCS00A102ZS9,RU000A102ZS9,1000.0,RISK_LEVEL_HIGH,rub,2023-10-10,2023-10-09,29.92
4,TCS00A102ZS9,RU000A102ZS9,1000.0,RISK_LEVEL_HIGH,rub,2023-07-11,2023-07-10,29.92
...,...,...,...,...,...,...,...,...
3553,TCS00A104735,RU000A104735,1000.0,RISK_LEVEL_MODERATE,rub,2023-12-07,2023-12-06,29.29
3554,TCS00A104735,RU000A104735,1000.0,RISK_LEVEL_MODERATE,rub,2023-09-07,2023-09-06,29.29
3555,TCS00A104735,RU000A104735,1000.0,RISK_LEVEL_MODERATE,rub,2023-06-08,2023-06-07,29.29
3556,BBG00J7HHGH1,RU000A0ZYG52,1000.0,RISK_LEVEL_LOW,rub,2023-11-14,2023-11-13,45.87


In [13]:
bonds_price_1d = info_bonds_hist_price(list(bonds_pay_info['figi'].unique()), '2023-03-31')
bonds_price_1d

,figi,data_candles,open,high,low,close
0,BBG00XH4W3N3,2023-03-31,80.330,80.330,79.700,79.960
1,TCS00A102D46,2023-03-31,88.200,88.200,88.200,88.200
2,BBG0116Q1YF0,2023-03-31,101.300,101.530,100.510,100.990
3,ISSUANCEPRLS,2023-03-31,100.000,100.090,100.000,100.090
4,TCS00A105WR1,2023-03-31,99.800,100.290,99.470,100.000
...,...,...,...,...,...,...
796,BBG00RRT3TX4,2023-03-31,98.954,98.954,98.804,98.922
797,TCS00A105HJ9,2023-03-31,100.010,100.290,99.660,100.290
798,TCS00A105DS9,2023-03-31,105.900,106.240,105.800,106.100
799,TCS00A104735,2023-03-31,96.750,96.750,96.420,96.730


In [14]:
bonds_pay_info = bonds_pay_info.merge(bonds_price_1d,
                                      how='left',
                                      on='figi')
bonds_pay_info

,figi,ticker,nominal,risk_level,currency,data,fix,pay,data_candles,open,high,low,close
0,BBG00XH4W3N3,RU000A101RZ3,1000.0,RISK_LEVEL_LOW,rub,2024-03-20,2024-03-19,34.90,2023-03-31,80.33,80.33,79.70,79.96
1,BBG00XH4W3N3,RU000A101RZ3,1000.0,RISK_LEVEL_LOW,rub,2023-09-20,2023-09-19,34.90,2023-03-31,80.33,80.33,79.70,79.96
2,TCS00A102ZS9,RU000A102ZS9,1000.0,RISK_LEVEL_HIGH,rub,2024-01-09,2023-12-29,29.92,NaN,NaN,NaN,NaN,NaN
3,TCS00A102ZS9,RU000A102ZS9,1000.0,RISK_LEVEL_HIGH,rub,2023-10-10,2023-10-09,29.92,NaN,NaN,NaN,NaN,NaN
4,TCS00A102ZS9,RU000A102ZS9,1000.0,RISK_LEVEL_HIGH,rub,2023-07-11,2023-07-10,29.92,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3553,TCS00A104735,RU000A104735,1000.0,RISK_LEVEL_MODERATE,rub,2023-12-07,2023-12-06,29.29,2023-03-31,96.75,96.75,96.42,96.73
3554,TCS00A104735,RU000A104735,1000.0,RISK_LEVEL_MODERATE,rub,2023-09-07,2023-09-06,29.29,2023-03-31,96.75,96.75,96.42,96.73
3555,TCS00A104735,RU000A104735,1000.0,RISK_LEVEL_MODERATE,rub,2023-06-08,2023-06-07,29.29,2023-03-31,96.75,96.75,96.42,96.73
3556,BBG00J7HHGH1,RU000A0ZYG52,1000.0,RISK_LEVEL_LOW,rub,2023-11-14,2023-11-13,45.87,2023-03-31,100.03,100.03,100.03,100.03


In [15]:
bonds_pay_info['open'] = bonds_pay_info['open'] / 100 * bonds_pay_info['nominal']
bonds_pay_info['high'] = bonds_pay_info['high'] / 100 * bonds_pay_info['nominal']
bonds_pay_info['low'] = bonds_pay_info['low'] / 100 * bonds_pay_info['nominal']
bonds_pay_info['close'] = bonds_pay_info['close'] / 100 * bonds_pay_info['nominal']
bonds_pay_info['risk_gr0'] = np.where(bonds_pay_info['risk_level'] == 'RISK_LEVEL_LOW',1,0)
bonds_pay_info['risk_gr1'] = np.where(bonds_pay_info['risk_level'].isin(['RISK_LEVEL_LOW', 'RISK_LEVEL_MODERATE']),1,0)
bonds_pay_info['risk_gr2'] = np.where(bonds_pay_info['risk_level'].isin(['RISK_LEVEL_LOW', 'RISK_LEVEL_MODERATE', 'RISK_LEVEL_HIGH']),1,0)

In [16]:
bonds_pay_info

,figi,ticker,nominal,risk_level,currency,data,fix,pay,data_candles,open,high,low,close,risk_gr0,risk_gr1,risk_gr2
0,BBG00XH4W3N3,RU000A101RZ3,1000.0,RISK_LEVEL_LOW,rub,2024-03-20,2024-03-19,34.90,2023-03-31,803.3,803.3,797.0,799.6,1,1,1
1,BBG00XH4W3N3,RU000A101RZ3,1000.0,RISK_LEVEL_LOW,rub,2023-09-20,2023-09-19,34.90,2023-03-31,803.3,803.3,797.0,799.6,1,1,1
2,TCS00A102ZS9,RU000A102ZS9,1000.0,RISK_LEVEL_HIGH,rub,2024-01-09,2023-12-29,29.92,NaN,NaN,NaN,NaN,NaN,0,0,1
3,TCS00A102ZS9,RU000A102ZS9,1000.0,RISK_LEVEL_HIGH,rub,2023-10-10,2023-10-09,29.92,NaN,NaN,NaN,NaN,NaN,0,0,1
4,TCS00A102ZS9,RU000A102ZS9,1000.0,RISK_LEVEL_HIGH,rub,2023-07-11,2023-07-10,29.92,NaN,NaN,NaN,NaN,NaN,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3553,TCS00A104735,RU000A104735,1000.0,RISK_LEVEL_MODERATE,rub,2023-12-07,2023-12-06,29.29,2023-03-31,967.5,967.5,964.2,967.3,0,1,1
3554,TCS00A104735,RU000A104735,1000.0,RISK_LEVEL_MODERATE,rub,2023-09-07,2023-09-06,29.29,2023-03-31,967.5,967.5,964.2,967.3,0,1,1
3555,TCS00A104735,RU000A104735,1000.0,RISK_LEVEL_MODERATE,rub,2023-06-08,2023-06-07,29.29,2023-03-31,967.5,967.5,964.2,967.3,0,1,1
3556,BBG00J7HHGH1,RU000A0ZYG52,1000.0,RISK_LEVEL_LOW,rub,2023-11-14,2023-11-13,45.87,2023-03-31,1000.3,1000.3,1000.3,1000.3,1,1,1


In [17]:
r_gr = bonds_pay_info.groupby('currency', as_index=False)\
                .agg({'fix':'nunique',
                      'pay':'mean',
                      'open':'mean'})

r_gr['all_pay'] = r_gr['fix'] * r_gr['pay'] * 0.87
r_gr

,currency,fix,pay,open,all_pay
0,rub,250,22.181151,896.147588,4824.400327


In [18]:
print(float(r_gr['all_pay']), 'Рублей купонами если покапать по 1 облигации перед каждой выплатой')
print(400000 / float(r_gr['all_pay']), 'Кол-во облигаций которые потребуются для получения вычета')
print(float(r_gr['all_pay']) * 20 * 0.13, 'Вычет если покупать 20 облигаций')

4824.4003267285 Рублей купонами если покапать по 1 облигации перед каждой выплатой
82.91185907269976 Кол-во облигаций которые потребуются для получения вычета
12543.4408494941 Вычет если покупать 20 облигаций


In [19]:
r_gr0 = bonds_pay_info[bonds_pay_info['risk_gr0']==1]\
                .groupby('currency', as_index=False)\
                .agg({'fix':'nunique',
                      'pay':'mean',
                      'open':'mean'})

r_gr0['all_pay'] = r_gr0['fix'] * r_gr0['pay'] * 0.87
r_gr0

,currency,fix,pay,open,all_pay
0,rub,227,25.341521,865.19083,5004.697071


In [20]:
print(float(r_gr0['all_pay']), 'Рублей купонами если покапать по 1 облигации перед каждой выплатой')
print(400000 / float(r_gr0['all_pay']), 'Кол-во облигаций которые потребуются для получения вычета')
print(float(r_gr0['all_pay']) * 20 * 0.13, 'Вычет если покупать 20 облигаций')

5004.697070710443 Рублей купонами если покапать по 1 облигации перед каждой выплатой
79.92491740228704 Кол-во облигаций которые потребуются для получения вычета
13012.21238384715 Вычет если покупать 20 облигаций


In [21]:
r_gr1 = bonds_pay_info[bonds_pay_info['risk_gr1']==1]\
                .groupby('currency', as_index=False)\
                .agg({'fix':'nunique',
                      'pay':'mean',
                      'open':'mean'})

r_gr1['all_pay'] = r_gr1['fix'] * r_gr1['pay'] * 0.87
r_gr1

,currency,fix,pay,open,all_pay
0,rub,248,25.671293,904.949345,5538.838227


In [22]:
print(float(r_gr1['all_pay']), 'Рублей купонами если покапать по 1 облигации перед каждой выплатой')
print(400000 / float(r_gr1['all_pay']), 'Кол-во облигаций которые потребуются для получения вычета')
print(float(r_gr1['all_pay']) * 20 * 0.13, 'Вычет если покупать 20 облигаций')

5538.838226801149 Рублей купонами если покапать по 1 облигации перед каждой выплатой
72.21731049383119 Кол-во облигаций которые потребуются для получения вычета
14400.979389682989 Вычет если покупать 20 облигаций


In [23]:
r_gr2 = bonds_pay_info[bonds_pay_info['risk_gr2']==1]\
                .groupby('currency', as_index=False)\
                .agg({'fix':'nunique',
                      'pay':'mean',
                      'open':'mean'})

r_gr2['all_pay'] = r_gr2['fix'] * r_gr2['pay'] * 0.87
r_gr2

,currency,fix,pay,open,all_pay
0,rub,250,21.728545,854.314659,4725.958489


In [24]:
print(float(r_gr2['all_pay']), 'Рублей купонами если покапать по 1 облигации перед каждой выплатой')
print(400000 / float(r_gr2['all_pay']), 'Кол-во облигаций которые потребуются для получения вычета')
print(float(r_gr2['all_pay']) * 20 * 0.13, 'Вычет если покупать 20 облигаций')

4725.958488805982 Рублей купонами если покапать по 1 облигации перед каждой выплатой
84.63891524808133 Кол-во облигаций которые потребуются для получения вычета
12287.492070895554 Вычет если покупать 20 облигаций


**Самые выгодные облигации для покупки с минимальным риском и средним риском**

1) Выгрузить историю выплаты купонов.  
2) Выгрузить цену за день фиксации реестра и через день (7 дней)  
3) Посмотреть изменение цен

In [25]:
df_coupons_2022 = info_coupons(list(bonds_rub_less_1000[bonds_rub_less_1000['risk_level'].isin(['RISK_LEVEL_MODERATE','RISK_LEVEL_LOW'])]['figi']), '2022-01-01', '2023-01-01')
df_coupons_2022

,figi,data,fix,pay
0,BBG00XH4W3N3,2022-09-21,2022-09-20,34.90
1,BBG00XH4W3N3,2022-03-23,2022-03-22,34.90
2,BBG00Z8KGGC3,2022-08-12,2022-08-11,31.16
3,BBG00Z8KGGC3,2022-02-11,2022-02-10,31.16
4,TCS00A102D46,2022-11-28,2022-11-25,8.45
...,...,...,...,...
1346,TCS00A104735,2022-09-08,2022-09-07,29.29
1347,TCS00A104735,2022-06-09,2022-06-08,29.29
1348,TCS00A104735,2022-03-10,2022-03-09,29.29
1349,BBG00J7HHGH1,2022-11-15,2022-11-14,38.39


In [26]:
bonds_rub_less_1000

,figi,ticker,nominal,risk_level,currency
0,BBG00XH4W3N3,RU000A101RZ3,1000.00,RISK_LEVEL_LOW,rub
2,TCS00A102ZS9,RU000A102ZS9,1000.00,RISK_LEVEL_HIGH,rub
3,BBG00Z8KGGC3,RU000A102R73,1000.00,RISK_LEVEL_LOW,rub
4,TCS00A102D46,RU000A102D46,518.11,RISK_LEVEL_LOW,rub
5,BBG0116Q1YF0,RU000A103455,1000.00,RISK_LEVEL_UNSPECIFIED,rub
...,...,...,...,...,...
1792,TCS00A105HJ9,RU000A105HJ9,1000.00,RISK_LEVEL_UNSPECIFIED,rub
1793,BBG00T7HZJG4,RU000A101L96,1000.00,RISK_LEVEL_MODERATE,rub
1794,TCS00A105DS9,RU000A105DS9,1000.00,RISK_LEVEL_MODERATE,rub
1795,TCS00A104735,RU000A104735,1000.00,RISK_LEVEL_MODERATE,rub


In [27]:
figi_h = []
data_h = []
open_h = []
high_h = []
low_h = []
close_h = []

fix_h = []

for i, row in df_coupons_2022.iterrows():
    f = row['figi']
    ds = str((datetime.datetime.strptime(row['fix'], '%Y-%m-%d')-datetime.timedelta(days=3)).date())
    de = str((datetime.datetime.strptime(row['fix'], '%Y-%m-%d')+datetime.timedelta(days=3)).date())
    gca1_h,gca2_h,gca3_h,gca4_h,gca5_h,gca6_h  = get_candles_day(f, ds, de)
    figi_h += gca1_h
    data_h += gca2_h
    open_h += gca3_h
    high_h += gca4_h
    low_h += gca5_h
    close_h += gca6_h
    fix_h += [row['fix']] * len(gca1_h)

price_before_after_coupons = pd.DataFrame({'figi':figi_h,
                                           'data_candles': data_h,
                                           'open': open_h,
                                           'high': high_h,
                                           'low':low_h,
                                           'close': close_h,
                                           'fix':fix_h})
price_before_after_coupons

,figi,data_candles,open,high,low,close,fix
0,BBG00XH4W3N3,2022-09-19,86.44,86.79,85.80,85.80,2022-09-20
1,BBG00XH4W3N3,2022-09-20,86.20,86.20,84.00,84.00,2022-09-20
2,BBG00XH4W3N3,2022-09-21,83.40,84.92,75.61,83.79,2022-09-20
3,BBG00XH4W3N3,2022-09-22,86.17,86.17,81.70,82.99,2022-09-20
4,BBG00XH4W3N3,2022-09-23,83.00,84.00,78.21,81.34,2022-09-20
...,...,...,...,...,...,...,...
5133,BBG00J7HHGH1,2022-05-13,95.08,98.00,95.08,97.75,2022-05-16
5134,BBG00J7HHGH1,2022-05-16,98.16,98.17,93.80,97.36,2022-05-16
5135,BBG00J7HHGH1,2022-05-17,97.95,97.95,96.90,97.63,2022-05-16
5136,BBG00J7HHGH1,2022-05-18,97.82,97.90,97.60,97.60,2022-05-16


In [28]:
price_before_after_coupons['data_candles'] = price_before_after_coupons['data_candles'].apply(lambda x:(datetime.datetime.strptime(x, '%Y-%m-%d')))
price_before_after_coupons['fix'] = price_before_after_coupons['fix'].apply(lambda x:(datetime.datetime.strptime(x, '%Y-%m-%d')))
price_before_after_coupons['days_to_fix'] = (price_before_after_coupons['data_candles'] - price_before_after_coupons['fix']).apply(lambda x: x.days)
price_before_after_coupons

,figi,data_candles,open,high,low,close,fix,days_to_fix
0,BBG00XH4W3N3,2022-09-19,86.44,86.79,85.80,85.80,2022-09-20,-1
1,BBG00XH4W3N3,2022-09-20,86.20,86.20,84.00,84.00,2022-09-20,0
2,BBG00XH4W3N3,2022-09-21,83.40,84.92,75.61,83.79,2022-09-20,1
3,BBG00XH4W3N3,2022-09-22,86.17,86.17,81.70,82.99,2022-09-20,2
4,BBG00XH4W3N3,2022-09-23,83.00,84.00,78.21,81.34,2022-09-20,3
...,...,...,...,...,...,...,...,...
5133,BBG00J7HHGH1,2022-05-13,95.08,98.00,95.08,97.75,2022-05-16,-3
5134,BBG00J7HHGH1,2022-05-16,98.16,98.17,93.80,97.36,2022-05-16,0
5135,BBG00J7HHGH1,2022-05-17,97.95,97.95,96.90,97.63,2022-05-16,1
5136,BBG00J7HHGH1,2022-05-18,97.82,97.90,97.60,97.60,2022-05-16,2


In [29]:
df_last_day_to_pay = price_before_after_coupons[price_before_after_coupons['days_to_fix']<0].groupby(['figi', 'fix'],as_index=False).agg({'days_to_fix':'max'})
df_first_day_to_sold = price_before_after_coupons[price_before_after_coupons['days_to_fix']>0].groupby(['figi', 'fix'],as_index=False).agg({'days_to_fix':'min'})

In [30]:
df_last_day_to_pay = df_last_day_to_pay.merge(price_before_after_coupons,
                                                  how='left',
                                                  on=['figi', 'fix','days_to_fix'])
df_last_day_to_pay = df_last_day_to_pay[['figi', 'fix', 'open', 'high', 'low', 'close']]
df_last_day_to_pay.columns=['figi', 'fix', 'open_l', 'high_l', 'low_l', 'close_l']
df_last_day_to_pay

,figi,fix,open_l,high_l,low_l,close_l
0,BBG000000006,2022-02-28,78.02,89.7700,78.0200,84.0000
1,BBG000000006,2022-05-30,90.79,91.7000,89.8600,91.4500
2,BBG000000006,2022-08-29,97.40,97.4800,97.0500,97.3900
3,BBG000000006,2022-11-28,97.92,97.9200,96.7300,97.3500
4,BBG00005V2L5,2021-12-22,97.67,97.6900,97.4600,97.6900
...,...,...,...,...,...,...
1043,TCS00A105518,2022-11-29,103.38,103.3800,101.0000,101.0000
1044,TCS00A1058K7,2022-12-26,103.38,103.3800,102.4000,102.4000
1045,TCS10A105146,2022-11-18,92.00,92.0000,91.5000,92.0000
1046,TCS188543193,2022-12-09,135.00,136.5099,134.7900,136.5099


In [31]:
df_first_day_to_sold = df_first_day_to_sold.merge(price_before_after_coupons,
                                                  how='left',
                                                  on=['figi', 'fix','days_to_fix'])
df_first_day_to_sold = df_first_day_to_sold[['figi', 'fix', 'open', 'high', 'low', 'close']]
df_first_day_to_sold.columns=['figi', 'fix', 'open_f', 'high_f', 'low_f', 'close_f']
df_first_day_to_sold

,figi,fix,open_f,high_f,low_f,close_f
0,BBG000000006,2022-05-30,90.8000,90.8000,89.1400,90.77
1,BBG000000006,2022-08-29,97.7800,97.7800,97.1000,97.40
2,BBG000000006,2022-11-28,97.5800,97.5800,97.1000,97.50
3,BBG0000776S2,2022-02-08,81.8220,83.6900,81.8210,83.50
4,BBG0000776S2,2022-08-09,87.1170,87.5340,86.5500,86.80
...,...,...,...,...,...,...
1009,TCS00A105518,2022-11-29,102.7000,103.0000,102.5000,102.50
1010,TCS00A1058K7,2022-12-26,103.2400,103.4000,102.5200,102.80
1011,TCS10A105146,2022-11-18,90.9405,92.4400,90.9405,91.98
1012,TCS188543193,2022-12-09,135.8000,136.7909,135.8000,136.09


In [62]:
df_f_l = df_last_day_to_pay.merge(df_first_day_to_sold,
                         how='inner',
                         on=['figi', 'fix'])
df_f_l = df_f_l.merge(bonds_rub_less_1000[['figi', 'nominal']],
                      how='inner',
                      on='figi')

df_f_l['loss_comm_004'] = (df_f_l['open_f'] / 100 * df_f_l['nominal'] * (1-0.0004)) - (df_f_l['open_l'] / 100 * df_f_l['nominal'] * (1.0004))
df_f_l['loss_comm_03'] = (df_f_l['open_f'] / 100 * df_f_l['nominal'] * (1-0.003)) - (df_f_l['open_l'] / 100 * df_f_l['nominal'] * (1.003)) 
df_f_l

,figi,fix,open_l,high_l,low_l,close_l,open_f,high_f,low_f,close_f,nominal,loss_comm_004,loss_comm_03
0,BBG000000006,2022-05-30,90.790,91.7000,89.8600,91.4500,90.8000,90.8000,89.1400,90.77,875.0,-0.548065,-4.679238
1,BBG000000006,2022-08-29,97.400,97.4800,97.0500,97.3900,97.7800,97.7800,97.1000,97.40,875.0,2.641870,-1.798475
2,BBG000000006,2022-11-28,97.920,97.9200,96.7300,97.3500,97.5800,97.5800,97.1000,97.50,875.0,-3.659250,-8.106875
3,BBG0000776S2,2022-02-08,81.951,83.6180,81.9510,83.4550,81.8220,83.6900,81.8210,83.50,1000.0,-1.945092,-6.203190
4,BBG0000776S2,2022-08-09,87.798,87.7980,86.3010,87.3400,87.1170,87.5340,86.5500,86.80,1000.0,-7.509660,-12.057450
...,...,...,...,...,...,...,...,...,...,...,...,...,...
978,TCS00A105518,2022-11-29,103.380,103.3800,101.0000,101.0000,102.7000,103.0000,102.5000,102.50,1000.0,-7.624320,-12.982400
979,TCS00A1058K7,2022-12-26,103.380,103.3800,102.4000,102.4000,103.2400,103.4000,102.5200,102.80,1000.0,-2.226480,-7.598600
980,TCS10A105146,2022-11-18,92.000,92.0000,91.5000,92.0000,90.9405,92.4400,90.9405,91.98,1000.0,-11.326762,-16.083215
981,TCS188543193,2022-12-09,135.000,136.5099,134.7900,136.5099,135.8000,136.7909,135.8000,136.09,1000.0,6.916800,-0.124000


In [64]:
print('Убыток от продажи облигации в последний торговый день и покупкой в первый торговый день после фиксации реестра', df_f_l['loss_comm_03'].mean())

Убыток от продажи облигации в последний торговый день и покупкой в первый торговый день после фиксации реестра -3.9612578996835235
